# Non linear analysis - Assignment 3

18.10.2023 - Jérémie Engler and Laure Toullier

In [2]:
#imports

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('bmh')
import math
import pandas as pd

np.set_printoptions(precision=5, suppress=True, linewidth=150)

Params = plt.rcParams
Params['figure.figsize'] = (14, 7) 

General data

In [58]:
E = 200e3 #Mpa
poisson = 0
sigma_y = 400 #Mpa

eps_0 = np.zeros((2,))
sigma_0 = np.zeros((2,))

strain_incr = np.array([0.003,0.004])

d = E/(1 - poisson**2) #Factor of the constitutive matrix

D = np.full((2,2), poisson)
np.fill_diagonal(D, 1)
D = D * d


In [84]:
def stress_incr_computation(strain_incr, plastic_strain_incr, n, D):
    if plastic_strain_incr.all() == 0:
        stress_incr = D @ strain_incr
    else:
        stress_incr = (D - (D @ n.T @ n @ D) / (n @ D @ n.T)) @ strain_incr

    return stress_incr

def check_failure(stress):
    fail = False
    criterion = stress[0]**2 + stress[1]**2 - stress[0] * stress[1] - sigma_y ** 2

    if criterion > 0:
        fail = True

    return fail

def strain_decomposition(strain_incr, alpha):
    elastic_strain_incr = alpha * strain_incr
    plastic_strain_incr = (1 - alpha) * strain_incr

    return elastic_strain_incr, plastic_strain_incr

def norm_criterion_surface(stress_y): #Unfinished
    n = np.array([[1,1]])
    return n

In [85]:
# Stress increment from strain increment

plastic_strain_incr = np.zeros((2,))
n = np.zeros((2,))

# First forward step

stress_incr = stress_incr_computation(strain_incr, plastic_strain_incr, n, D)

stress = sigma_0 + stress_incr

if check_failure(stress) == True:
    alpha = np.sqrt((sigma_y**2) / (stress[0]**2 + stress[1]**2 - stress[0] * stress[1]))
    stress_y = alpha * stress

elastic_strain_incr, plastic_strain_incr = strain_decomposition(strain_incr, alpha)

# Second forward step

sub_increments = 2

plastic_strain_incr_2 = plastic_strain_incr / sub_increments

n = norm_criterion_surface(stress_y)

stress_incr = stress_incr_computation(strain_incr, plastic_strain_incr_2, n, D)

print(stress_incr)


[-100.  100.]


[300. 400.]
